In [5]:
"""
레코드, 필드로 구성된 여러개의 테이블
SQL : DBMS와 대화하는 표준 방법, 쿼리 : SQL문으로 DBMS에 데이터를 요청하는것
접근 : SELECT
SELECT * FROM 테이블명 테이블명의 모든 데이터를 읽고 싶을 때 사용, *은 모든 열이다.
SELECT 열이름 FROM 테이블명 : 테이블명에 있는 열이름에 해당하는 열만 데려온다.
SELECT book_name FROM table_c WHERE pub_year=2022 table_c에서 pub_year이 2022인 book_name만 데려온다.
삽입 : INSERT
INSERT INTO table_d (book_name, pub_year) VALUES ("혼공 머신러닝", 2020) table_d에 book_name열에 혼공 머신러닝을 pub_year열에 2020을 삽입한다.
나열순서를 잘 지켜야한다.
CREATE TABLE로 테이블 만들기
CREATE TABLE nslib_book (name TEXT, author TEXT, borrow_count INTEGER) : nslib_book 테이블을 만들고 name,author,borrow_count 열을 만든다.
"""
import sqlite3
# 1. 데이터베이스를 connect()로 빈 데이터베이스를 만들어야함.
conn = sqlite3.connect('ns_lib.db')

import gdown
gdown.download('https://bit.ly/3RhoNho', 'ns_202104.csv', quiet=False)

# 2. SQL문 실행하기 : cursor(), execute()
c = conn.cursor() # 커서 지정
c.execute("CREATE TABLE IF NOT EXISTS nslib_book (name TEXT,author TEXT,borrow_count INTEGER)") # 해당 커서(파일과 연결됨)로 명령 실행, 파일이 이미 있을 수도 있으니 IF NOT EXISTS를 쓴다.

import pandas as pd
ns_df = pd.read_csv('ns_202104.csv', low_memory=False)
ns_df.head()
# 한행씩 반복 실행하기 : for문과 iterrows() : 한행씩 순환하는 명령어이고 인덱스와 행객체를 반환한다. -> 한행씩 순환하는게 최적화가 안되어있어서 느림.
# for index,row in ns_df.iterrows():
#   c.execute("INSERT INTO nslib_book (name,author,borrow_count) VALUES (?,?,?)", (row['도서명'],row['저자'],row['대출건수']))

# to_sql()로 데이터프레임 데이터를 테이블에 추가하기
book_df = ns_df[['도서명','저자','대출건수']]
book_df.columns = ['name','author','borrow_count'] # columns 속성으로 열 이름 바꿔주기
book_df.to_sql('nslib_book', conn, if_exists='replace', index=False) # 사용할 데이터베이스 테이블 이름, 커넥션 객체, if_exists는 fail,replace,append가 있다.
# fail은 이미 있으면 오류, replace는 새걸로 대체, append는 걍 옆에 데이터를 추가한다.

# 파이썬으로 테이블 데이터 읽기
c.execute("SELECT * FROM nslib_book")
print(c.fetchone()) # 한행씩 읽기
print(c.fetchmany(10)) # 10행씩 읽기
# print(c.fetchall()) # 모든 행 출력
all_rows = c.fetchall()
book_df = pd.DataFrame(all_rows)
print(book_df.head())

# 데이터베이스를 읽어 데이터 프레임을 만들기 : read_sql_query(), 테이블을 데이터프레임으로 읽기 : read_sql_table()
book_df = pd.read_sql_query("SELECT * FROM nslib_book", conn)
print(book_df.head()) # 열 이름을 데이터프레임의 열 이름으로 자동 사용

# 통계량 구하기
len(book_df) # 401682
c.execute("SELECT COUNT(*) FROM nslib_book") # count(*)로 열의 값이 있는 행의 개수 세기. *는 모든 열이니 모든 행의 개수를 세준다.
print(c.fetchone())

c.execute("SELECT sum(borrow_count) FROM nslib_book") # 빌린 횟수의 합
print(c.fetchone())

c.execute("SELECT avg(borrow_count) FROM nslib_book") # 빌린 횟수의 평균
print(c.fetchone())

# 데이터 정렬하기 : ORDER BY
c.execute("SELECT * FROM nslib_book ORDER BY borrow_count DESC") # borrow_count 내림차순을 따라 정렬한다. DESC가 없으면 기본 오름차순이다.
print(c.fetchall())

# 자동으로 닫히긴 하지만 데이터베이스 사용을 마칠 때 커서와 커넥션 객체를 명시적으로 닫아주는 것이 좋다.
c.close()
conn.close()

Downloading...
From: https://bit.ly/3RhoNho
To: /content/ns_202104.csv
100%|██████████| 57.6M/57.6M [00:00<00:00, 185MB/s]


('인공지능과 흙', '김동훈 지음', 0)
[('가짜 행복 권하는 사회', '김태형 지음', 0), ('나도 한 문장 잘 쓰면 바랄 게 없겠네', '김선영 지음', 0), ('예루살렘 해변', '이도 게펜 지음, 임재희 옮김', 0), ('김성곤의 중국한시기행 : 장강·황하 편', '김성곤 지음', 0), ('처음 읽는 음식의 세계사', '미야자키 마사카츠 지음, 한세희 옮김', 0), ('아르센 벵거 자서전 My Life in Red and White', '아르센 벵거 지음, 이성모 옮김', 0), ('쓰고 달콤한 직업', '천운영 지음', 0), ('일잘러의 노션 100% 활용법', '제레미 강 지음', 0), ('타이난 골목 노포 산책', '천구이팡 지음, 심혜경 외 옮김', 0), ('이디스 워튼의 환상 이야기', '이디스 워튼 지음, 성소희 옮김', 0)]
             0                     1  2
0        질서 너머  조던 B. 피터슨 지음, 김한영 옮김  0
1  성장을 꿈꾸는 너에게         우쥔 지음, 이지수 옮김  0
2       스티븐 호킹  레너드 믈로디노프 지음, 하인해 옮김  0
3       벌의 사생활      소어 핸슨 지음, 하윤숙 옮김  0
4     지브리의 천재들    스즈키 도시오 지음, 이선희 옮김  0
                    name            author  borrow_count
0                인공지능과 흙            김동훈 지음             0
1           가짜 행복 권하는 사회            김태형 지음             0
2  나도 한 문장 잘 쓰면 바랄 게 없겠네            김선영 지음             0
3                예루살렘 해변  이도 게펜 지음, 임재희 옮김             0
4  김성곤의 중국한시기행 : 장강·황하 편            김성곤 